# Pegasus Quickstart

Welcome to the Pegasus Quickstart tutorial notebook, which is intended for new users who want to get a quick overview of Pegasus concepts and usage. This tutorial covers:

 - Using the Pegasus API to generate an abstract workflow
 - Using the API to plan the abstract workflow into an executable workflow
 - Run the workflow first locally, and then have the jobs in the workflow run on remote ACCESS resources.
 
For a quick overview of Pegasus, please see this short YouTube video:

[![A 5 Minute Introduction](../images/youtube-pegasus-intro.png)](https://www.youtube.com/watch?v=MNN80OHMQUQ "A 5 Minute Introduction")


## Hello World Workflow

This notebook will generate a simple **Hello World** workflow illustrated below, then plan and execute the workflow. We will first run this workflow locally on the ACCESS Pegasus host, and then run on the remote ACCESS resources.

Rectangles represent input/output files, and ovals represent compute jobs. The arrows represent file dependencies between each compute job. This simple workflow will execute 2 jobs, each taking in one input file and generating a single output file. The **world** job is dependant on the output of the **hello**.

Each job in this workflow invokes a python executable named *hello* and *world* . These are symlinks to the same executable *pegasus-keg.py* . We use symbolic links in this case, to have the executable and the job name match. The python code is a simple code that takes in an input file; captures the hostname of the node where it is run on; and also includes the contents of the input file in it's output.

![Hello World Workflow](../images/pipeline.svg)

The abstract workflow description that you specify to Pegasus is portable, and usually does not contain any locations to physical input files, executables or cluster end points where jobs are executed. 

In this example, Pegasus will 

* pick up the inputs from a directory named **input** .
* pick up the executables from a directory named **executables** .
* place the generated outputs in a directory named **output** .


<div class="alert alert-block alert-info">
<b>Note:</b> There is a separation between the environment where this notebook runs and where the compute job executes. This notebook runs on pegasus.access-ci.org, while the job is executed on any available HTCondor execution points. For this tutorial, a small number of execution points will be provided automatically. For larger workflows, you will learn in the `Provisioning` tutorial how to allocate additional resources using your allocations.
</div>


## 1. Import Python API

Pegasus 5.0 introduces a new Python API, which is fully documented in the [Pegasus reference guide](https://pegasus.isi.edu/documentation/reference-guide/api-reference.html). 

We will mainly use 3 main classes in this simple quickstart example

<br>

```
from Pegasus.api.replica_catalog import File
from Pegasus.api.workflow import Job, Workflow
from Pegasus.client._client import PegasusClientError
```

The `Workflow` object is used to store jobs and dependencies between each job. Typical job creation is as follows:

```
# Define job Input/Output files
input_file = File("input.txt")
output_file1 = File("output1.txt")
output_file2 = File("output2.txt")

# Define job, passing in the transformation (executable) it will use
j = Job(transformation_obj)

# Specify command line arguments (if any) which will be passed to the transformation when run
j.add_args("arg1", "arg2", input_file, "arg3", output_file)

# Specify input files (if any)
j.add_inputs(input_file)

# Specify output files (if any)
j.add_outputs(output_file1, output_file2)

# Add profiles to the job
j.add_env(FOO="bar")
j.add_profiles(Namespace.PEGASUS, key="checkpoint.time", value=1)

# Add the job to the workflow object
wf.add_jobs(j)
```

By default, dependencies between jobs are inferred based on input and output files. 

In [ ]:
from Pegasus.api import *
import sys
from pathlib import Path

import logging

logging.basicConfig(level=logging.DEBUG)

# we specify directories for inputs, executables and outputs
# - directory where to pick up the inputs from a directory.
# - directory where the executables that the workflow uses are placed.
# - directory where the outputs should be placed.

BASE_DIR = Path(".").resolve()
INPUT_DIR = Path(BASE_DIR /  "input").resolve()
EXECUTABLES_DIR = Path(BASE_DIR / ".." /  "executables").resolve()
OUTPUT_DIR = Path(BASE_DIR /  "output").resolve() 

# the execution site where you job to run.
# local means the jobs run on ACCESS Pegasus itself.
# condorpool means jobs will run on a node provisioned from an ACCESS site such as jetstream
EXEC_SITE="local"

# generate a simple input file for the workflow
with open("{}/f.in".format(INPUT_DIR), "w") as f:
    f.write("This is the contents of the input file for the hello world workflow!")

# --- Workflow -----------------------------------------------------------------
wf = Workflow("hello-world")

fin = File("f.in")
finter = File("f.inter")
fout = File("f.out")

job_hello = Job("hello")\
                    .add_args("-T", "3", "-i", fin, "-o {}".format(finter))\
                    .add_inputs(fin)\
                    .add_outputs(finter)

job_world = Job("world")\
                    .add_args("-T", "3", "-i", finter, "-o {}".format(fout))\
                    .add_inputs(finter)\
                    .add_outputs(fout)

wf.add_jobs(job_hello, job_world)    

# --- Visualize the Workflow ---------------------------------------------------
try:
    wf.write()
    wf.graph(include_files=True, label="xform-id", output="graph.png")
except PegasusClientError as e:
    print(e)

In [ ]:
# view rendered workflow
from IPython.display import Image
Image(filename='graph.png')

## 2. Run the Workflow

When working in Python, we can just use the reference do the `Workflow` object, you can plan, run, and monitor the workflow directly. These are wrappers around Pegasus CLI tools, and as such, the same arguments may be passed to them. 

In [ ]:
try:
    wf.plan(input_dirs=[INPUT_DIR], sites=[EXEC_SITE], transformations_dir=EXECUTABLES_DIR,\
            output_dir=OUTPUT_DIR, submit=True)
except PegasusClientError as e:
    print(e)


Note the line in the output that starts with pegasus-status, contains the command you can use to monitor the status of the workflow. We will cover this command line tool in the next couple of notbooks. The path it contains is the path to the submit directory where all of the files required to submit and monitor the workflow are stored. For now we will just continue to use the Python `Workflow` object

In [ ]:
wf.status(long=True)

Wait for the workflow to finish, and then display the results

We can also just block on the workflow finishing:

In [ ]:
wf.wait()

## 3. Inspecting the generated output of the workflow

The executable that is run as part of this worklfow, is a simple python script that captures the hostname, where
a job ran, and also includes the contents of the input file in it's output.

The Hostname in the .out file indicates that the jobs ran on pegasus.access-ci.org. 


In [ ]:
! cat output/f.out

##  4. Statistics

Depending on if the workflow finished successfully or not, you have options on what to do next. If the workflow failed you can use `wf.analyze()` do get help finding out what went wrong. If the workflow finished successfully, we can pull out some statistcs from the provenance database:

In [ ]:
try:
    wf.statistics()
except PegasusClientError as e:
    print(e)

##  5. Run the jobs in the workflow on ACCESS resources

Since we planned the workflow for a site named **local** the jobs ran on ACCESS Pegasus host itself.

Now we will plan the same workflow again and have it run on a site named **condorpool**.

Note, that we are not re-defining the workflow. The workflow to Pegasus is described in a portable resource agnostic way. We are taking the same abstract workflow and now, will replan it for a different execution environment.

<div class="alert alert-block alert-info">
<b>Note:</b> There is a separation between the environment where this notebook runs and where the compute job executes. This notebook runs on pegasus.access-ci.org, while the job is executed on any available HTCondor execution points. For this tutorial, a small number of execution points will be provided automatically. For larger workflows, you will learn in the `Provisioning` tutorial how to allocate additional resources using your allocations.
The following figure shows how workflows are defined using the Pegasus API in Jupyter, planned to an executable
HTCondor DAGMan, and jobs flow to the remote execution sites (TestPool in this case).
</div>

<img src="../images/access-pegasus-jobflow.png"/>

In [ ]:
try:
    wf.plan(input_dirs=[INPUT_DIR], sites=["condorpool"], transformations_dir=EXECUTABLES_DIR,\
            output_dir=OUTPUT_DIR, submit=True)\
      .wait()
except PegasusClientError as e:
    print(e)

### 5.1 Reinspecting the output of the workflow 

Now lets review the output of the workflow to check where the jobs ran. The displayed hostname will have a prefix of  testpool-cpu-* . The test pool is made of nodes provisioned from the Indiana University's **Jetstream2** which is a Cloud ACCESS resource.

In [ ]:
! cat output/f.out